# Spatial functions - 1

## Measurements - area

In [ ]:
%matplotlib inline
import geopandas as gpd

buowl = gpd.read_file("data/BUOWL_Habitat.shp")
raptors = gpd.read_file("data/Raptor_Nests.shp")
linears = gpd.read_file("data/Linear_Projects.shp")

If we are going to work with measurements then we need to convert to a projection suitable for calculating areas and lengths.

EPSG:2163 is the US National Atlas equal area projection which covers the entire US

If the area is small enough UTM zones will likely be more accurate

In [ ]:
buowl = buowl.to_crs(epsg=2163)
buowl.crs

Create a new field showing the area calculated with crs = 2163

In [ ]:
buowl['area_2163']=buowl['geometry'].area/10000
buowl.head(10)

Convert the GeoDataFrame to UTM Zone 13 coordinates

In [ ]:
buowl = buowl.to_crs(epsg=26913)

Now create a new field showing the area when projected to UTM zone 13 

In [ ]:
buowl['area_26913']=buowl['geometry'].area/10000
buowl.head()

Now lets create a field showing the difference in areas measured as a percentage and check the range of differences.

In [ ]:
buowl['area_diff']=(buowl['area_26913']-buowl['area_2163'])/buowl['area_2163']*100
buowl.describe()

## Measurements - length and perimeter

length can be calculated simply by calling the length method on a GeoSeries

In [ ]:
buowl['perimeter'] = buowl['geometry'].length
buowl.head()

Now lets calculate what the minimum perimeter of a polygon with the same area could be.  This would be a circle of the same area.

To calculate the perimeter of a circle with a given area we have to use the formula for the area of a circle $A = \pi r^2$ and solve for the radius $ r=\sqrt{\frac{A}{\pi} }$ then we can use the radius to calculate the circumference of a circle of that area $C=2\pi r$

In [ ]:
buowl['min_perimeter'] = (buowl['geometry'].area/3.14159)**0.5*3.14159*2
buowl.head()

Now just for fun lets calculate the ratio of the perimeter of a polygon to the perimeter of a circle with the same area.

The lowest possible value would be 1 which would be a circle.

Higher values indicate more complex shapes

In [ ]:
buowl["perimeter_ratio"] = buowl['perimeter']/buowl['min_perimeter']
buowl.head()

And lets sort the burrowing owl habitat polygons in descending order

In [ ]:
buowl.sort_values('perimeter_ratio', ascending=False)

The polygon with the most complex shape has a habitat_id of 323. Lets look at its shape.

In [ ]:
buowl[buowl['habitat_id']==378].plot()

Now lets calculate the length of some linear features

In [ ]:
linears.crs

The linears GeoDataFrame is projected with geographic coordinates (latitude and longitude) which are not suitable for calculating measurements.  

We could of course reproject the entire GeoDataFrame like we did with burrowing owls but what if we want to leave the GeoDataFrame in geographic coordinates and still get a length in meters?

In [ ]:
linears['length']=linears['geometry'].to_crs(epsg=26913).length
linears

Just for fun lets calculate the area and length of a point GeoDataFrame

In [ ]:
raptors['area']=raptors['geometry'].area
raptors['length']=raptors['geometry'].length
raptors